In [ ]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fb344d5631fd38994e76ca613dca9a5c84c0e20e1c3cff65206ce26c3833714663d7a57036891ac56cd74a3e151b23654006ac978993e2cce'

## --- Day 9: Disk Fragmenter ---
Another push of the button leaves you in the familiar hallways of some friendly amphipods! Good thing you each somehow got your own personal mini submarine. The Historians jet away in search of the Chief, mostly by driving directly into walls.

While The Historians quickly figure out how to pilot these things, you notice an amphipod in the corner struggling with his computer. He's trying to make more contiguous free space by compacting all of the files, but his program isn't working; you offer to help.

He shows you the disk map (your puzzle input) he's already generated. For example:
```
2333133121414131402
```
The disk map uses a dense format to represent the layout of files and free space on the disk. The digits alternate between indicating the length of a file and the length of free space.

So, a disk map like 12345 would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like 90909 would represent three nine-block files in a row (with no free space between them).

Each file on disk also has an ID number based on the order of the files as they appear before they are rearranged, starting with ID 0. So, the disk map 12345 has three files: a one-block file with ID 0, a three-block file with ID 1, and a five-block file with ID 2. Using one character for each block where digits are the file ID and . is free space, the disk map 12345 represents these individual blocks:
```
0..111....22222
```
The first example above, 2333133121414131402, represents these individual blocks:
```
00...111...2...333.44.5555.6666.777.888899
```
The amphipod would like to move file blocks one at a time from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map 12345, the process looks like this:
```
0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
```
The first example requires a few more steps:
```
00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
```
The final step of this file-compacting process is to update the filesystem checksum. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position 0. If a block contains free space, skip it instead.

Continuing the first example, the first few blocks' position multiplied by its file ID number are 0 * 0 = 0, 1 * 0 = 0, 2 * 9 = 18, 3 * 9 = 27, 4 * 8 = 32, and so on. In this example, the checksum is the sum of these, 1928.

Compact the amphipod's hard drive using the process he requested. What is the resulting filesystem checksum? (Be careful copy/pasting the input for this puzzle; it is a single, very long line.)

In [ ]:
def convert_disk_map(line):
    tot=0
    diskmap = ""
    for index, fs in enumerate(line):
        file_id, is_file = divmod(index, 2)
        fs = int(fs)
        if is_file==0:
            diskmap += "".join(str(file_id) for i in range(fs))
        else:
            diskmap += "".join("." for i in range(fs))
        tot+=fs
    print(f"Converting len line={len(line)} len diskmap={len(diskmap)} tot={tot}")

    return diskmap

In [ ]:
convert_disk_map("12345")

Converting len line=5 len diskmap=15 tot=15


'0..111....22222'

In [ ]:
convert_disk_map("2333133121414131402")

Converting len line=19 len diskmap=42 tot=42


'00...111...2...333.44.5555.6666.777.888899'

In [ ]:
convert_disk_map("23331331214141314022333133121414131402")

Converting len line=38 len diskmap=98 tot=84


'00...111...2...333.44.5555.6666.777.888899..101010...111111.121212...13..14....15....16...17......'

In [ ]:
from aocd import get_data
aoc_input = get_data(day=9, year=2024)

In [ ]:
def calculate_checksum(diskmap):
    checksum = 0
    for pos, block in enumerate(diskmap):
        if block != '.':  # skip free spaces
            file_id = int(block)
            checksum += pos * file_id
    return checksum

In [ ]:
def defragment_disk(input_line): 
    # Convert input to disk map and then to list
    converted = convert_disk_map(input_line)
    print("starting dm:", converted)
    diskmap = list(converted)
       
    # Keep track of the next empty space
    empty_pointer = 0
    while empty_pointer < len(diskmap) and diskmap[empty_pointer] != '.':
        empty_pointer += 1
    
    # Defragmentation process
    for rindex in range(len(diskmap)-1, -1, -1):
        fileblock = diskmap[rindex]
        if fileblock != '.' and empty_pointer < rindex:
            # Move the block
            diskmap[empty_pointer] = fileblock
            diskmap[rindex] = "."
            
            # Find next empty space
            empty_pointer += 1
            while empty_pointer < len(diskmap) and diskmap[empty_pointer] != '.':
                empty_pointer += 1
    
    final_disk = ''.join(diskmap)
    checksum = calculate_checksum(final_disk)
    
    return final_disk, checksum

In [ ]:
def test(test2=False):
    line = "12345"  # result is 0..111....22222
    expected_dm = "022111222......"
    expected_checksum = 60

    if test2:
        sample_input="2333133121414131402"
        line = sample_input
        expected_dm = "0099811188827773336446555566.............."
        expected_checksum = 1928

    diskmap, checksum = defragment_disk(line)

    print(f"Match: {diskmap == expected_dm}")  # Changed expected to expected_dm
    print(f"Length: {len(diskmap)} vs {len(expected_dm)}")  # Changed expected to expected_dm
    print("Checksum =", checksum)
    if diskmap != expected_dm:
        print("Actual:   ", diskmap)
        print("Expected: ", expected_dm)
    if checksum != expected_checksum:
        print("Actual:   ", checksum)
        print("Expected: ", expected_checksum)

In [ ]:
test()
test(True)

Converting len line=5 len diskmap=15 tot=15
starting dm: 0..111....22222
Match: True
Length: 15 vs 15
Checksum = 60
Converting len line=19 len diskmap=42 tot=42
starting dm: 00...111...2...333.44.5555.6666.777.888899
Match: True
Length: 42 vs 42
Checksum = 1928


In [ ]:
diskmap, checksum = defragment_disk(aoc_input)
print("Checksum =", checksum)

Converting len line=19999 len diskmap=238882 tot=94613
starting dm: 000.......111122222222..334.....5555555........6.......77........88888...91010101010.11...1212121212121212.1313131313.....1414141414..15151515151515151616161616.......17171717171717....1818........1919191919191919........202020202020202020.....212121212121212121........2222.....23.........242525.26262626.......27272727.........28....2929292929292929........3030303030303030....3131313131.......32......3333333333333333........343434343434..35353535353535......36363636......37....3838383838383838..393939...40...4141414141.......42424243434343434343....4444444444444444..454545454545454545......46464646464646......47474747..484848484848.......49494949.....5050........515151515151.........525252525252.......535353535353535353...545454545454545454.....55555555555555.......565656...5757575757575757...585959....60606060..6161616161....626262626262...636363636363636363..6464646464646465656565.....6666......6767676767676768686868

## working uptil here but we get wrong answer on input data

**AI Prompt**

ok i think i figured out the issue - its in the convert_disk_map. when we have fileid go over 9, then we basically end up writing 2 digit file ids ... and we don't account for that in our later functions. how do we modify the defragment disk function to account for 2 digit file ids

**AI Response**

That's a great observation! You're right - when file IDs become double digits, it changes how we need to handle the blocks. Let me ask a few questions to help guide our solution:

1. How do you want to handle this? We have two main options:
   - We could use a delimiter between numbers
   - We could modify our data structure to store blocks differently (like using a list of integers instead of a string)

Which approach would you prefer to explore first? I can help you think through the pros and cons of each.

**AI Prompt**

talk to me about pros and cons of each approach

**AI Response**

Let's break down each approach:

Using a delimiter (e.g., commas or spaces between numbers):
Pros:
- We can keep most of our existing code structure
- Easy to visualize the disk state when printing
- Simple to parse numbers since they're clearly separated

Cons:
- Need to handle the delimiter carefully when moving blocks
- Might make the string manipulation more complex
- Takes more space in memory

Using a list of integers:
Pros:
- Clean representation of file IDs (no string parsing needed)
- No ambiguity between single and double-digit numbers
- More efficient operations when moving blocks
- Natural Python list operations available

Cons:
- Need to significantly modify existing code
- Might need separate handling for empty spaces
- Different printing format needed for visualization

Would you like to explore the implementation details of either approach?

**AI Prompt**

ok lets try approach 1 ie using a delimiter. can u share the code modifications we would need to make ?

**AI Response**

I'll help guide you through modifying the code to use delimiters. Let's start with the `convert_disk_map` function since that's where we first create our disk representation.

Would you like me to show you how to modify this function first? We could use something like '|' as a delimiter between blocks, and '.' for empty spaces.

Before I write any code, would you prefer to:
1. Try modifying the function yourself with this hint
2. See my suggested modification and then work on the rest yourself
3. Have me help you through each step of the modifications

What would be most helpful for you?

**AI Prompt**

yeah just show me the modified code

**AI Response**

Here's the modified `convert_disk_map` function using '|' as a delimiter:

```python
def convert_disk_map(line):
    tot = 0
    diskmap = []
    for index, fs in enumerate(line):
        file_id, is_file = divmod(index, 2)
        fs = int(fs)
        if is_file == 0:
            diskmap.extend([f"{file_id}|" for _ in range(fs)])
        else:
            diskmap.extend([".|" for _ in range(fs)])
        tot += fs
    print(f"Converting len line={len(line)} len diskmap={len(diskmap)} tot={tot}")
    return "".join(diskmap)
```

Would you like to test this first before we modify the other functions? We should make sure this produces the expected output format before moving forward.

In [ ]:
def convert_disk_map(line):
    tot = 0
    diskmap = []
    for index, fs in enumerate(line):
        file_id, is_file = divmod(index, 2)
        fs = int(fs)
        if is_file == 0:
            diskmap.extend([f"{file_id}|" for _ in range(fs)])
        else:
            diskmap.extend([".|" for _ in range(fs)])
        tot += fs
    print(f"Converting len line={len(line)} len diskmap={len(diskmap)} tot={tot}")
    return "".join(diskmap)

In [ ]:
convert_disk_map("23331331214141314022333133121414131402")

Converting len line=38 len diskmap=84 tot=84


'0|0|.|.|.|1|1|1|.|.|.|2|.|.|.|3|3|3|.|4|4|.|5|5|5|5|.|6|6|6|6|.|7|7|7|.|8|8|8|8|9|9|.|.|10|10|10|.|.|.|11|11|11|.|12|12|12|.|.|.|13|.|.|14|.|.|.|.|15|.|.|.|.|16|.|.|.|17|.|.|.|.|.|.|'

**AI Prompt**

ok can u show me the modifed code now for the rest

**AI Response**

Here are the modified `calculate_checksum` and `defragment_disk` functions to work with the delimiter format:

```python
def calculate_checksum(diskmap):
    checksum = 0
    blocks = diskmap.split('|')[:-1]  # Split by delimiter, ignore last empty element
    for pos, block in enumerate(blocks):
        if block != '.':  # skip free spaces
            file_id = int(block)
            checksum += pos * file_id
    return checksum

def defragment_disk(input_line): 
    converted = convert_disk_map(input_line)
    print("starting dm:", converted)
    blocks = converted.split('|')[:-1]  # Split into blocks, ignore last empty element
    
    # Keep track of the next empty space
    empty_pointer = 0
    while empty_pointer < len(blocks) and blocks[empty_pointer] != '.':
        empty_pointer += 1
    
    # Defragmentation process
    for rindex in range(len(blocks)-1, -1, -1):
        fileblock = blocks[rindex]
        if fileblock != '.' and empty_pointer < rindex:
            # Move the block
            blocks[empty_pointer] = fileblock
            blocks[rindex] = "."
            
            # Find next empty space
            empty_pointer += 1
            while empty_pointer < len(blocks) and blocks[empty_pointer] != '.':
                empty_pointer += 1
    
    final_disk = '|'.join(blocks) + '|'  # Add back the delimiters
    checksum = calculate_checksum(final_disk)
    
    return final_disk, checksum
```

Would you like to test this with our sample inputs first to make sure it's working correctly?

In [ ]:
def calculate_checksum(diskmap):
    checksum = 0
    blocks = diskmap.split('|')[:-1]  # Split by delimiter, ignore last empty element
    for pos, block in enumerate(blocks):
        if block != '.':  # skip free spaces
            file_id = int(block)
            checksum += pos * file_id
    return checksum

def defragment_disk(input_line): 
    converted = convert_disk_map(input_line)
    print("starting dm:", converted)
    blocks = converted.split('|')[:-1]  # Split into blocks, ignore last empty element
    
    # Keep track of the next empty space
    empty_pointer = 0
    while empty_pointer < len(blocks) and blocks[empty_pointer] != '.':
        empty_pointer += 1
    
    # Defragmentation process
    for rindex in range(len(blocks)-1, -1, -1):
        fileblock = blocks[rindex]
        if fileblock != '.' and empty_pointer < rindex:
            # Move the block
            blocks[empty_pointer] = fileblock
            blocks[rindex] = "."
            
            # Find next empty space
            empty_pointer += 1
            while empty_pointer < len(blocks) and blocks[empty_pointer] != '.':
                empty_pointer += 1
    
    final_disk = '|'.join(blocks) + '|'  # Add back the delimiters
    checksum = calculate_checksum(final_disk)
    
    return ''.join(blocks), checksum

In [ ]:
test()
test(True)

Converting len line=5 len diskmap=15 tot=15
starting dm: 0|.|.|1|1|1|.|.|.|.|2|2|2|2|2|
Match: True
Length: 15 vs 15
Checksum = 60
Converting len line=19 len diskmap=42 tot=42
starting dm: 0|0|.|.|.|1|1|1|.|.|.|2|.|.|.|3|3|3|.|4|4|.|5|5|5|5|.|6|6|6|6|.|7|7|7|.|8|8|8|8|9|9|
Match: True
Length: 42 vs 42
Checksum = 1928


In [ ]:
diskmap, checksum = defragment_disk(aoc_input)
print("Checksum =", checksum)

Converting len line=19999 len diskmap=94613 tot=94613
starting dm: 0|0|0|.|.|.|.|.|.|.|1|1|1|1|2|2|2|2|2|2|2|2|.|.|3|3|4|.|.|.|.|.|5|5|5|5|5|5|5|.|.|.|.|.|.|.|.|6|.|.|.|.|.|.|.|7|7|.|.|.|.|.|.|.|.|8|8|8|8|8|.|.|.|9|10|10|10|10|10|.|11|.|.|.|12|12|12|12|12|12|12|12|.|13|13|13|13|13|.|.|.|.|.|14|14|14|14|14|.|.|15|15|15|15|15|15|15|15|16|16|16|16|16|.|.|.|.|.|.|.|17|17|17|17|17|17|17|.|.|.|.|18|18|.|.|.|.|.|.|.|.|19|19|19|19|19|19|19|19|.|.|.|.|.|.|.|.|20|20|20|20|20|20|20|20|20|.|.|.|.|.|21|21|21|21|21|21|21|21|21|.|.|.|.|.|.|.|.|22|22|.|.|.|.|.|23|.|.|.|.|.|.|.|.|.|24|25|25|.|26|26|26|26|.|.|.|.|.|.|.|27|27|27|27|.|.|.|.|.|.|.|.|.|28|.|.|.|.|29|29|29|29|29|29|29|29|.|.|.|.|.|.|.|.|30|30|30|30|30|30|30|30|.|.|.|.|31|31|31|31|31|.|.|.|.|.|.|.|32|.|.|.|.|.|.|33|33|33|33|33|33|33|33|.|.|.|.|.|.|.|.|34|34|34|34|34|34|.|.|35|35|35|35|35|35|35|.|.|.|.|.|.|36|36|36|36|.|.|.|.|.|.|37|.|.|.|.|38|38|38|38|38|38|38|38|.|.|39|39|39|.|.|.|40|.|.|.|41|41|41|41|41|.|.|.|.|.|.|.|42|42|42|43|43|43|43|43

## --- Part Two ---
Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

The first example from above now proceeds differently:
```
00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
```
The process of updating the filesystem checksum is the same; now, this example's checksum would be 2858.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?

In [ ]:
# we change delimiters to appear after every whole file block now 
def convert_disk_map(line):
    tot = 0
    diskmap = []
    for index, fs in enumerate(line):
        file_id, is_file = divmod(index, 2)
        fs = int(fs)
        if is_file == 0:
            diskmap.extend([f"{file_id}" for _ in range(fs)])
            diskmap.extend(["|"])
        else:
            diskmap.extend(["." for _ in range(fs)])
            diskmap.extend(["|"])
        tot += fs
    print(f"Converting len line={len(line)} len diskmap={len(diskmap)} tot={tot}")
    return "".join(diskmap)

In [ ]:
print(convert_disk_map("12345"))
print(convert_disk_map("2333133121414131402"))
print(convert_disk_map("233313312141413140223"))

Converting len line=5 len diskmap=20 tot=15
0|..|111|....|22222|
Converting len line=19 len diskmap=61 tot=42
00|...|111|...|2|...|333|.|44|.|5555|.|6666|.|777|.|8888||99|
Converting len line=21 len diskmap=68 tot=47
00|...|111|...|2|...|333|.|44|.|5555|.|6666|.|777|.|8888||99|..|101010|


In [ ]:
def defragment_disk(input_line): 
    converted = convert_disk_map(input_line)
    print("starting dm:", converted)
    blocks = converted.split('|')[:-1]  # Split into file blocks, ignore last empty element

    # Keep track of the next empty space
    empty_pointer = 0
    while empty_pointer < len(blocks) and blocks[empty_pointer][0] != '.':
        empty_pointer += 1
    print(f"ep={empty_pointer}")

    # Defragmentation process
    for rindex in range(len(blocks)-1, -1, -1):
        fileblock = blocks[rindex]
        print(f"fileblock={fileblock}, {blocks[empty_pointer]}")
        lfb = len(fileblock)
        lep = len(blocks[empty_pointer])
        rem_empty = lep-lfb
        if fileblock[0] != '.' and empty_pointer < rindex and rem_empty>=0:
            print(f"moving the block")
            # Move the block
            blocks[empty_pointer] = fileblock 
            # insert new block of length rem_empty 
            blocks[rindex] = "."*lfb
            blocks.insert(empty_pointer + 1, "." * rem_empty)                       
            print(f"block now is:{blocks}")

            
            # Find next empty space
            empty_pointer += 1
            while empty_pointer < len(blocks) and blocks[empty_pointer] != '.':
                empty_pointer += 1
            print(f"new ep={empty_pointer}")
    
    final_disk = '|'.join(blocks) + '|'  # Add back the delimiters
    #checksum = calculate_checksum(final_disk)
    return final_disk
    #return ''.join(blocks), checksum

In [ ]:
fd = defragment_disk("12345")

Converting len line=5 len diskmap=20 tot=15
starting dm: 0|..|111|....|22222|
ep=1
fileblock=22222, ..
fileblock=...., ..
fileblock=111, ..
fileblock=.., ..
fileblock=0, ..


In [ ]:
fd

'0|..|111|....|22222|'

In [ ]:
fd = defragment_disk("2333133121414131402")
fd

Converting len line=19 len diskmap=61 tot=42
starting dm: 00|...|111|...|2|...|333|.|44|.|5555|.|6666|.|777|.|8888||99|
ep=1
fileblock=99, ...
moving the block
block now is:['00', '99', '.', '111', '...', '2', '...', '333', '.', '44', '.', '5555', '.', '6666', '.', '777', '.', '8888', '', '..']
new ep=2
fileblock=8888, .
fileblock=., .
fileblock=777, .
fileblock=., .
fileblock=6666, .
fileblock=., .
fileblock=5555, .
fileblock=., .
fileblock=44, .
fileblock=., .
fileblock=333, .
fileblock=..., .
fileblock=2, .
moving the block
block now is:['00', '99', '2', '', '111', '...', '.', '...', '333', '.', '44', '.', '5555', '.', '6666', '.', '777', '.', '8888', '', '..']
new ep=6
fileblock=111, .
fileblock=, .
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[1], line 1
----> 1 fd = defragment_disk("2333133121414131402")
      2 fd

Cell In[1], line 19, in defragment_disk(input_line)


IndexError: string index out of range

In [ ]:
converted = convert_disk_map("12345")
blocks = converted.split('|')[:-1] 
empty_pointer = 0
while empty_pointer < len(blocks) and blocks[empty_pointer] != '.':
    empty_pointer += 1
print(f"ep={empty_pointer}")

Converting len line=5 len diskmap=20 tot=15
ep=5


In [ ]:
blocks

['0', '..', '111', '....', '22222']

In [ ]:
blocks[1][0]=='.'

True

**AI Prompt**

how do i modify my function to insert the rem_empty "." block into the blocks list ?

**AI Response**

To insert the remaining empty space as a new block, you'll need to handle the case where you have leftover space after moving a file. Here's how you could modify that part of the code:

```python
if fileblock[0] != '.' and empty_pointer < rindex and rem_empty>=0:
    # Move the block
    blocks[empty_pointer] = fileblock 
    # If there's remaining empty space, create a new empty block
    if rem_empty > 0:
        blocks[empty_pointer] += "." * rem_empty
    blocks[rindex] = "." * lfb
```

Or if you want to keep the blocks separate:

```python
if fileblock[0] != '.' and empty_pointer < rindex and rem_empty>=0:
    # Move the block
    blocks[empty_pointer] = fileblock 
    # If there's remaining empty space, insert a new block
    if rem_empty > 0:
        blocks.insert(empty_pointer + 1, "." * rem_empty)
    blocks[rindex] = "." * lfb
```

Which approach would you prefer to try first? The first one keeps the space in the same block, while the second one creates a separate block for the remaining space.

**AI Prompt**

can u help debug the overall function ?

**AI Response**

Let's debug the function step by step. First, let's modify the function to include more debug prints so we can see what's happening at each step. Here's a version with detailed logging:

```python
def defragment_disk(input_line): 
    converted = convert_disk_map(input_line)
    print("\nStarting disk map:", converted)
    blocks = converted.split('|')[:-1]
    print("Initial blocks:", blocks)

    # Keep track of the next empty space
    empty_pointer = 0
    while empty_pointer < len(blocks) and not blocks[empty_pointer].startswith('.'):
        empty_pointer += 1
    print(f"Initial empty pointer at position: {empty_pointer}")

    # Defragmentation process
    for rindex in range(len(blocks)-1, -1, -1):
        fileblock = blocks[rindex]
        print(f"\nProcessing block at position {rindex}: '{fileblock}'")
        
        if empty_pointer >= len(blocks):
            print("No more empty spaces available")
            break
            
        print(f"Current empty block at {empty_pointer}: '{blocks[empty_pointer]}'")
        
        if not fileblock.startswith('.') and empty_pointer < rindex:
            lfb = len(fileblock)
            lep = len(blocks[empty_pointer])
            rem_empty = lep - lfb
            print(f"File block length: {lfb}, Empty space length: {lep}, Remaining space: {rem_empty}")
            
            if rem_empty >= 0:
                print(f"Moving '{fileblock}' to position {empty_pointer}")
                blocks[empty_pointer] = fileblock
                if rem_empty > 0:
                    print(f"Inserting {rem_empty} empty spaces after moved block")
                    blocks.insert(empty_pointer + 1, "." * rem_empty)
                blocks[rindex] = "." * lfb
                print("Current blocks:", blocks)
                
                # Find next empty space
                empty_pointer += 1
                while empty_pointer < len(blocks) and not blocks[empty_pointer].startswith('.'):
                    empty_pointer += 1
                print(f"New empty pointer position: {empty_pointer}")
            else:
                print("Not enough space to move this block")
    
    final_disk = '|'.join(blocks) + '|'
    print("\nFinal disk map:", final_disk)
    return final_disk
```

Would you like to run this with a simple test case first to see where things might be going wrong?

In [ ]:
def defragment_disk(input_line): 
    converted = convert_disk_map(input_line)
    print("\nStarting disk map:", converted)
    blocks = converted.split('|')[:-1]
    print("Initial blocks:", blocks)

    # Keep track of the next empty space
    empty_pointer = 0
    while empty_pointer < len(blocks) and not blocks[empty_pointer].startswith('.'):
        empty_pointer += 1
    print(f"Initial empty pointer at position: {empty_pointer}")

    # Defragmentation process
    for rindex in range(len(blocks)-1, -1, -1):
        fileblock = blocks[rindex]
        print(f"\nProcessing block at position {rindex}: '{fileblock}'")
        
        if empty_pointer >= len(blocks):
            print("No more empty spaces available")
            break
            
        print(f"Current empty block at {empty_pointer}: '{blocks[empty_pointer]}'")
        
        if not fileblock.startswith('.') and empty_pointer < rindex:
            lfb = len(fileblock)
            lep = len(blocks[empty_pointer])
            rem_empty = lep - lfb
            print(f"File block length: {lfb}, Empty space length: {lep}, Remaining space: {rem_empty}")
            
            if rem_empty >= 0:
                print(f"Moving '{fileblock}' to position {empty_pointer}")
                blocks[empty_pointer] = fileblock
                if rem_empty > 0:
                    print(f"Inserting {rem_empty} empty spaces after moved block")
                    blocks.insert(empty_pointer + 1, "." * rem_empty)
                blocks[rindex] = "." * lfb
                print("Current blocks:", blocks)
                
                # Find next empty space
                empty_pointer += 1
                while empty_pointer < len(blocks) and not blocks[empty_pointer].startswith('.'):
                    empty_pointer += 1
                print(f"New empty pointer position: {empty_pointer}")
            else:
                print("Not enough space to move this block")
    
    final_disk = '|'.join(blocks) + '|'
    print("\nFinal disk map:", final_disk)
    return final_disk

In [ ]:
fd = defragment_disk("12345")

Converting len line=5 len diskmap=20 tot=15

Starting disk map: 0|..|111|....|22222|
Initial blocks: ['0', '..', '111', '....', '22222']
Initial empty pointer at position: 1

Processing block at position 4: '22222'
Current empty block at 1: '..'
File block length: 5, Empty space length: 2, Remaining space: -3
Not enough space to move this block

Processing block at position 3: '....'
Current empty block at 1: '..'

Processing block at position 2: '111'
Current empty block at 1: '..'
File block length: 3, Empty space length: 2, Remaining space: -1
Not enough space to move this block

Processing block at position 1: '..'
Current empty block at 1: '..'

Processing block at position 0: '0'
Current empty block at 1: '..'

Final disk map: 0|..|111|....|22222|


In [ ]:
fd = defragment_disk("2333133121414131402")
fd

Converting len line=19 len diskmap=61 tot=42

Starting disk map: 00|...|111|...|2|...|333|.|44|.|5555|.|6666|.|777|.|8888||99|
Initial blocks: ['00', '...', '111', '...', '2', '...', '333', '.', '44', '.', '5555', '.', '6666', '.', '777', '.', '8888', '', '99']
Initial empty pointer at position: 1

Processing block at position 18: '99'
Current empty block at 1: '...'
File block length: 2, Empty space length: 3, Remaining space: 1
Moving '99' to position 1
Inserting 1 empty spaces after moved block
Current blocks: ['00', '99', '.', '111', '...', '2', '...', '333', '.', '44', '.', '5555', '.', '6666', '.', '777', '.', '8888', '..', '99']
New empty pointer position: 2

Processing block at position 17: '8888'
Current empty block at 2: '.'
File block length: 4, Empty space length: 1, Remaining space: -3
Not enough space to move this block

Processing block at position 16: '.'
Current empty block at 2: '.'

Processing block at position 15: '777'
Current empty block at 2: '.'
File block lengt

'00|99|2|111|...|.|...|333|.|44|.|5555|.|6666|.|777|.|8888|..|99|'

In [ ]:
def find_empty_pointer(blocks, plen):
    for index, block in enumerate(blocks):
        if block.startswith('.') and len(block)<=plen:
            return index

def defragment_disk(input_line): 
    converted = convert_disk_map(input_line)
    print("\nStarting disk map:", converted)
    blocks = converted.split('|')[:-1]
    print("Initial blocks:", blocks)

    # Defragmentation process
    for rindex in range(len(blocks)-1, -1, -1):
        fileblock = blocks[rindex]
        print(f"\nProcessing block at position {rindex}: '{fileblock}'")
        
        empty_pointer = find_empty_pointer(blocks, len(fileblock))
        '''
        if empty_pointer >= len(blocks):
            print("No more empty spaces available")
            break
        '''
        print(f"Current empty block at {empty_pointer}: '{blocks[empty_pointer]}'")
        
        if not fileblock.startswith('.') and empty_pointer < rindex:
            lfb = len(fileblock)
            lep = len(blocks[empty_pointer])
            rem_empty = lep - lfb
            print(f"File block length: {lfb}, Empty space length: {lep}, Remaining space: {rem_empty}")
            
            if rem_empty >= 0:
                print(f"Moving '{fileblock}' to position {empty_pointer}")
                blocks[empty_pointer] = fileblock
                if rem_empty > 0:
                    print(f"Inserting {rem_empty} empty spaces after moved block")
                    blocks.insert(empty_pointer + 1, "." * rem_empty)
                blocks[rindex] = "." * lfb
                print("Current blocks:", blocks)
            else:
                print("Not enough space to move this block")
    
    final_disk = '|'.join(blocks) + '|'
    print("\nFinal disk map:", final_disk)
    return final_disk

In [ ]:
fd = defragment_disk("12345")
fd

Converting len line=5 len diskmap=20 tot=15

Starting disk map: 0|..|111|....|22222|
Initial blocks: ['0', '..', '111', '....', '22222']

Processing block at position 4: '22222'
Current empty block at 1: '..'
File block length: 5, Empty space length: 2, Remaining space: -3
Not enough space to move this block

Processing block at position 3: '....'
Current empty block at 1: '..'

Processing block at position 2: '111'
Current empty block at 1: '..'
File block length: 3, Empty space length: 2, Remaining space: -1
Not enough space to move this block

Processing block at position 1: '..'
Current empty block at 1: '..'

Processing block at position 0: '0'
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[1], line 1
----> 1 fd = defragment_disk("12345")
      2 fd

Cell In[1], line 23, in defragment_disk(input_line)
     17 empty_pointer = find_empty_pointer(blocks, len(fileblock))
  

TypeError: list indices must be integers or slices, not NoneType

**AI Prompt**

lets brainstorm different ways to attack this problem

**AI Response**

Let's break down different approaches we could take to solve this disk defragmentation problem:

1. Array/List Based Approaches:
- Keep track of files and spaces separately in two lists
- Track each block's size and type (file or space)
- Could make movement operations simpler

2. String Manipulation Approach:
- Work with the string representation directly
- Use regex or string operations to find and move blocks
- Might be easier for visualization

3. Object-Oriented Approach:
- Create File and Space objects
- Each object knows its size and position
- Could make the logic clearer

4. Graph-Based Approach:
- Treat files and spaces as nodes
- Edges represent possible moves
- Could help find optimal movements

Which of these approaches interests you most? We can explore the pros and cons of any of them in more detail.

**AI Prompt**

can u list out pros and cons of each approach

**AI Response**

I'll break down the pros and cons of each approach:

1. Array/List Based Approach:
Pros:
- Simple data structure
- Easy to manipulate blocks with list operations
- Good performance for sequential operations
- Straightforward to track sizes and positions

Cons:
- Need to handle block splitting carefully
- Could get complex when managing multiple lists
- May need additional bookkeeping for file IDs

2. String Manipulation Approach:
Pros:
- Natural representation of the disk state
- Easy to visualize and debug
- Good for pattern matching operations
- Simple to maintain format of input/output

Cons:
- String operations can be slow
- More complex to handle multi-digit file IDs
- Harder to track block sizes dynamically

Would you like me to continue with the other two approaches, or would you like to discuss any of these points in more detail first?